In [1]:
import pandas as pd
import seaborn as sns

pd.set_option('display.max_columns', 10)
pd.set_option('display.max_colwidth', 20)

titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age', 'sex', 'class', 'fare', 'survived']]

df.head()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


# 피봇테이블 만들기

## 각 클래스 별로 평균나이를 성별을 나누어서 구한다.

In [3]:
pdf1=pd.pivot_table(df,           # 피봇할 데이터 프레앰
                   index='class', # 행 위치에 들어갈 행
                   columns='sex', # 열 위치에 들어갈 열
                   values='age',  # 데이터로 사용할 열
                   aggfunc='mean')# 데이터 집계 함수.

In [4]:
pdf1

sex,female,male
class,,
First,34.611765,41.281386
Second,28.722973,30.740707
Third,21.750000,26.507589


## 직접해보기
- 각클래스 별로 생존율과 생존인원수 남녀구별하여구함.

In [15]:
pdf2=pd.pivot_table(df,           # 피봇할 데이터 프레앰
                   index='class', # 행 위치에 들어갈 행
                   columns='sex', # 열 위치에 들어갈 열
                   values='survived',  # 데이터로 사용할 열
                   aggfunc=['mean','sum'])# 데이터 집계 함수.

In [16]:
pdf2

mean              sum     
sex       female      male female male
class                                 
First   0.968085  0.368852     91   45
Second  0.921053  0.157407     70   17
Third   0.500000  0.135447     72   47

- 각 클래스와 성별로 나이와 요금의 평균과 최대값을 구한다.

In [23]:
pdf3=pd.pivot_table(df,           # 피봇할 데이터 프레앰
                   index=['class','sex'], # 행 위치에 들어갈 행
                   columns='survived', # 열 위치에 들어갈 열
                   values=['age','fare'],  # 데이터로 사용할 열
                   aggfunc=['mean','max'])# 데이터 집계 함수.

In [24]:
pdf3

mean                                      max        \
                     age                   fare               age         
survived               0          1           0           1     0     1   
class  sex                                                                
First  female  25.666667  34.939024  110.604167  105.978159  50.0  63.0   
       male    44.581967  36.248000   62.894910   74.637320  71.0  80.0   
Second female  36.000000  28.080882   18.250000   22.288989  57.0  55.0   
       male    33.369048  16.022000   19.488965   21.095100  70.0  62.0   
Third  female  23.818182  19.329787   19.773093   12.464526  48.0  63.0   
       male    27.255814  22.274211   12.204469   15.579696  74.0  45.0   

                                 
                 fare            
survived            0         1  
class  sex                       
First  female  151.55  512.3292  
       male    263.00  512.3292  
Second female   26.00   65.0000  
       male     73.50   39.0000  
Third  female   69.55   31.3875  
       male     69.55   56.4958

## 멀티인덱스와 멀티컬럼

In [25]:
pdf3.index

MultiIndex([( 'First', 'female'),
            ( 'First',   'male'),
            ('Second', 'female'),
            ('Second',   'male'),
            ( 'Third', 'female'),
            ( 'Third',   'male')],
           names=['class', 'sex'])

In [26]:
pdf3.columns

MultiIndex([('mean',  'age', 0),
            ('mean',  'age', 1),
            ('mean', 'fare', 0),
            ('mean', 'fare', 1),
            ( 'max',  'age', 0),
            ( 'max',  'age', 1),
            ( 'max', 'fare', 0),
            ( 'max', 'fare', 1)],
           names=[None, None, 'survived'])

# 피봇테이블에 접근하기 : xs인덱스를 사용하자

## 인덱스를 기준으로 접근하기

### 하나의 인덱스를 통한 접근

In [28]:
pdf3.xs('First')

mean                                      max                \
                age                   fare               age          fare   
survived          0          1           0           1     0     1       0   
sex                                                                          
female    25.666667  34.939024  110.604167  105.978159  50.0  63.0  151.55   
male      44.581967  36.248000   62.894910   74.637320  71.0  80.0  263.00   

                    
                    
survived         1  
sex                 
female    512.3292  
male      512.3292

### 여러개의 인덱스를 통한 접근

In [32]:
pdf3.xs(('First','female'))

            survived
mean  age   0            25.666667
            1            34.939024
      fare  0           110.604167
            1           105.978159
max   age   0            50.000000
            1            63.000000
      fare  0           151.550000
            1           512.329200
Name: (First, female), dtype: float64

In [33]:
pd.DataFrame(pdf3.xs(('First','female')))

First
                        female
          survived            
mean age  0          25.666667
          1          34.939024
     fare 0         110.604167
          1         105.978159
max  age  0          50.000000
          1          63.000000
     fare 0         151.550000
          1         512.329200

### 기준인덱스를 level로 특정할수있다

In [34]:
pdf3.xs('male',level='sex')

mean                                    max                \
                age                  fare              age          fare   
survived          0          1          0          1     0     1       0   
class                                                                      
First     44.581967  36.248000  62.894910  74.637320  71.0  80.0  263.00   
Second    33.369048  16.022000  19.488965  21.095100  70.0  62.0   73.50   
Third     27.255814  22.274211  12.204469  15.579696  74.0  45.0   69.55   

                    
                    
survived         1  
class               
First     512.3292  
Second     39.0000  
Third      56.4958

In [37]:
a = pdf3.xs(('Second', 'male'))
b = pdf3.xs(('Second', 'male'), level=[0, 'sex'])

In [39]:
print(type(a))
print(type(b))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [40]:
a

            survived
mean  age   0           33.369048
            1           16.022000
      fare  0           19.488965
            1           21.095100
max   age   0           70.000000
            1           62.000000
      fare  0           73.500000
            1           39.000000
Name: (Second, male), dtype: float64

In [41]:
b

mean                               max                  
                   age               fare            age        fare      
survived             0       1          0        1     0     1     0     1
class  sex                                                                
Second male  33.369048  16.022  19.488965  21.0951  70.0  62.0  73.5  39.0

## 열로선택하기

### 하나의 열을 기준으로 선택하기

In [43]:
pdf3.xs('mean',axis=1)

age                   fare            
survived               0          1           0           1
class  sex                                                 
First  female  25.666667  34.939024  110.604167  105.978159
       male    44.581967  36.248000   62.894910   74.637320
Second female  36.000000  28.080882   18.250000   22.288989
       male    33.369048  16.022000   19.488965   21.095100
Third  female  23.818182  19.329787   19.773093   12.464526
       male    27.255814  22.274211   12.204469   15.579696

In [44]:
pdf3.xs(('mean','age'),axis=1)

survived               0          1
class  sex                         
First  female  25.666667  34.939024
       male    44.581967  36.248000
Second female  36.000000  28.080882
       male    33.369048  16.022000
Third  female  23.818182  19.329787
       male    27.255814  22.274211

In [48]:
pdf3.xs(1,level='survived',axis=1)

mean               max          
                     age        fare   age      fare
class  sex                                          
First  female  34.939024  105.978159  63.0  512.3292
       male    36.248000   74.637320  80.0  512.3292
Second female  28.080882   22.288989  55.0   65.0000
       male    16.022000   21.095100  62.0   39.0000
Third  female  19.329787   12.464526  63.0   31.3875
       male    22.274211   15.579696  45.0   56.4958

In [49]:
pdf3.xs(('max','fare',0), level=[0,1,2],axis=1)

max
                 fare
survived            0
class  sex           
First  female  151.55
       male    263.00
Second female   26.00
       male     73.50
Third  female   69.55
       male     69.55